In [358]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans 
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.decomposition import NMF
from collections import defaultdict
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
#from nltk.corpus import stopwords
from sklearn.feature_extraction import stop_words

In [359]:
data = pd.read_csv('data/data_not_starbucks.csv').drop('Unnamed: 0', axis=1)

In [360]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474 entries, 0 to 473
Data columns (total 6 columns):
location.lat        474 non-null float64
location.lng        474 non-null float64
name                474 non-null object
combined_reviews    474 non-null object
num_review_words    474 non-null int64
final_address       474 non-null object
dtypes: float64(2), int64(1), object(3)
memory usage: 22.3+ KB


### Filter out shops with less than 20 words in their review

In [361]:
over20_data = data[data['num_review_words'] >= 20].reset_index().drop('index', axis=1)

### Adding custom stopwords to sklearns defaults

In [362]:
stopwords = list(stop_words.ENGLISH_STOP_WORDS)

In [363]:
stopwords += ['coffee', 'shop', 'coffeeshop', 'starbucks', 'wa', 'seattle']

### Added LemmaTokenizer to lemmatize words 

In [364]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [365]:
tf = TfidfVectorizer(strip_accents='unicode',
                     tokenizer=LemmaTokenizer(),
                     stop_words=stopwords,
                     max_features=500)

In [366]:
tfidf = tf.fit_transform(over20_data['combined_reviews'])

In [367]:
words = tf.get_feature_names()

In [406]:
nmf = NMF(n_components=40)
nmf.fit(tfidf)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=40, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [407]:
H = nmf.components_
W = nmf.transform(tfidf)

In [408]:
H.shape

(40, 500)

In [409]:
W.shape

(435, 40)

In [410]:
top_words_index = np.argsort(-H)[:,0:8]
most_common_words_per_topic = np.array(words)[top_words_index]
for i, items in enumerate(most_common_words_per_topic):
    print(i, items)

0 ['good' 'staff' 'neighborhood' 'spot' 'small' 'drip' 'little' 'morning']
1 ['sandwich' 'breakfast' 'delicious' 'turkey' 'egg' 'panini' 'snack' 'yum']
2 ['mocha' 'best' 'orange' 'try' 'foam' 'game' 'caffe' 'cafe']
3 ['bagel' 'cheese' 'cream' 'sandwich' 'breakfast' 'egg' 'good' 'really']
4 ['crepe' 'like' 'wanted' 'stop' 'neighborhood' 'instead' 'strawberry'
 'regular']
5 ['cupcake' 'cake' 'velvet' 'red' 'free' 'salted' 'delicious' 'bakery']
6 ['service' 'location' 'customer' 'right' 'convenient' 'place' 'order'
 'rude']
7 ['beer' 'wine' 'good' 'selection' 'night' 'work' 'vibe' 'cool']
8 ['donut' 'doughnut' 'fashioned' 'pot' 'old' 'maple' 'pumpkin' 'cake']
9 ['croissant' 'pastry' 'bakery' 'baked' 'good' 'macaroon' 'almond' 'bread']
10 ['chocolate' 'hot' 'dark' 'favorite' 'chip' 'cookie' 'rich' 'order']
11 ['great' 'love' 'service' 'place' 'people' 'awesome' 'customer' 'super']
12 ['owner' 'business' 'local' 'neighborhood' 'space' 'coming' 'ha' 'enjoy']
13 ['latte' 'tea' 'try' 'art' 'ch

In [398]:
#def top_shops(W):
shop_dict = defaultdict(list)
for index, item in enumerate(W):
    key = np.argmax(item)
    value = item[key]
    name = over20_data['name'][index]
    shop_dict[key].append([value, name])
top_shop_names = defaultdict(list)
for feature in shop_dict:
    top_shop_names[feature] = list(np.sort(np.array(shop_dict[feature]).T)[1,-6:-1])
top_shop_names

defaultdict(list,
            {0: ['Fuel Coffee Seattle',
              'Green Bean Coffeehouse',
              'My Coffeehouse',
              'QED Coffee',
              'Tougo Coffee Co.'],
             1: ['Palazzo Cafe',
              'The Station',
              'Treehouse Coffee',
              'Trinity Market',
              'World Class Coffee'],
             2: ['Dilettante Mocha Café - Rainier Square',
              'Ghost Alley Espresso',
              'Phoenix Cafe',
              'Uptown Espresso',
              'Uptown Espresso Gameporium  - Delridge'],
             3: ['Einstein Bros Bagels',
              'Einstein Bros Bagels',
              'Fat Ducks Deli & Bakery',
              'Grateful Bread Baking Company & Cafe',
              "Lama G's"],
             4: ['Cafe Javasti',
              'Citizen Cafe',
              'Essential Bakery Cafe',
              'Joe Bar',
              'Le Petite Cafe'],
             5: ['Cubes Baking Co.',
              'Cupcake Roya

In [412]:
0.8/(0.7+0.8)

0.5333333333333333

In [415]:
data.sort_values?

In [416]:
pd.Dataframe?

Object `pd.Dataframe` not found.


In [417]:
data

,location.lat,location.lng,name,combined_reviews,num_review_words,final_address
0,47.579130,-122.410511,Alki Cafe,if you cansit in hespers section she gives exc...,369,2726 Alki Ave SW
1,47.579352,-122.409126,Tully's Coffee,in addition to the great coffee theres a pleas...,536,2676 Alki Ave SW
2,47.580447,-122.406728,Ampersand Cafe,craft coffee drinks cider beer wine try the o...,339,2536 Alki Ave SW
3,47.680561,-122.404709,Jibe Espresso Bar,sandwiches and pastries made in house with a v...,362,7001 Seaview Ave NW #170
4,47.675598,-122.398264,The Scoop at Walter's,snoqualmie ice cream decent espresso and appar...,126,6408 32nd Ave NW
5,47.675674,-122.398387,Pico Café,various breads baked inhouse available daily p...,112,6415 32nd Ave NW
6,47.675592,-122.398240,Walter's,i love this place everybody is so nice in the...,196,6408 32nd Ave NW
7,47.659875,-122.398059,Discovery Espresso,roasted pasilla chile breakfast sammy is aweso...,164,3103 W Jameson St
8,47.639363,-122.399467,Uptown Espresso Magnolia,their famous velvet foam really is velvet get ...,379,3223 W McGraw St
9,47.668781,-122.391557,Firehouse Coffee,employees are very nice i used to go here ver...,549,2622 NW Market St


In [418]:
new_df = pd.DataFrame?

In [435]:
new_df = data[['location.lat', 'location.lng', 'name', 'num_review_words']]

In [436]:
list(new_df.iloc[0:2]['name'])

['Alki Cafe', "Tully's Coffee"]

In [439]:
new_df = new_df.sort_values('num_review_words', ascending=False)

In [440]:
new_df

,location.lat,location.lng,name,num_review_words
305,47.614044,-122.328289,Starbucks Reserve Roastery & Tasting Room,3327
172,47.615175,-122.341202,Top Pot Doughnuts,2887
374,47.612082,-122.317028,Café Presse,2120
176,47.609821,-122.342205,Le Panier,2033
194,47.609010,-122.339658,Seattle Coffee Works,1805
138,47.649376,-122.347683,Milstead & Co.,1743
348,47.623752,-122.320998,Espresso Vivace,1703
154,47.682145,-122.344709,Beth's Café,1669
140,47.625605,-122.346250,Citizen Cafe,1655
177,47.608968,-122.340697,Storyville Coffee Company,1635
